<a href="https://colab.research.google.com/github/youiba/BERT_Model_For_Sentiment_Analysis/blob/main/Bert1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras
from keras import datasets, Model
from keras.layers import Dense, Flatten
from transformers import BertModel, BertTokenizer, BertConfig, TFAutoModelForSequenceClassification

In [ ]:
(X_train , y_train) , (X_test , y_test) = keras.datasets.imdb.load_data()

In [ ]:
X_train_new = X_train[0:100]
y_train_new = y_train[0:100]
X_test_new = X_test[0:100]
y_test_new = y_test[0:100]

In [ ]:
tz = BertTokenizer.from_pretrained("bert-base-cased")

In [ ]:
input_ids = []
token_type_ids = []
attention_masks = []

In [ ]:
# Encode the sentence
for sentence in X_train_new:
    encoded = tz.encode_plus(
        text=sentence,  # the sentence to be encoded
        add_special_tokens=True,  # Add [CLS] and [SEP]
        pad_to_max_length=True,  # Add [PAD]s
        max_length = 300,
        padding='max_length',
        truncation=True,
        return_attention_mask = True  # Generate the attention mask
    )
    input_ids.append(encoded['input_ids'])
    token_type_ids.append(encoded['token_type_ids'])
    attention_masks.append(encoded['attention_mask'])


In [ ]:
print(X_train_new[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [ ]:
print(input_ids[0])

[101, 1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
vocab_dict = keras.datasets.imdb.get_word_index()
sent = []
for embedding in input_ids[0]:
  if embedding == 101:
    word = "[CLS]"
  elif embedding == 102:
    word = "[SEP]"
  else:
    if word not in vocab_dict:
      word = "[OOV]"
    else:
      word = vocab_dict[embedding]
  sent.append(word)
print(sent)

['[CLS]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', '[OOV]', 

In [ ]:
input_ids = np.array(input_ids)
token_type_ids = np.array(token_type_ids)
attention_masks = np.array(attention_masks)

print(input_ids.shape)
print(token_type_ids.shape)
print(attention_masks.shape)

(100, 300)
(100, 300)
(100, 300)


In [ ]:
#checkpoint = 'bert-base-cased'
#model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
max_seq_length = 300
input_layer_1 = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_layer_2 = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
input_layer_3 = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_layer_1, input_layer_2, input_layer_3])
flattn_lyr = tf.keras.layers.Flatten()(pooled_output)
dense_1 = tf.keras.layers.Dense(512, activation="relu")(flattn_lyr)
output_layer = tf.keras.layers.Dense(2, activation="softmax", name="classification")(dense_1)

In [ ]:
model = tf.keras.Model(inputs=[input_layer_1, input_layer_2, input_layer_3], outputs=[output_layer])

In [ ]:
loss= keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer='adam', loss=loss, metrics='Accuracy')

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 300)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 300)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 300)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       [(None, 768),        109482241   ['input_word_ids[0][0]',         
                                 (None, 300, 768)]                'input_mask[0][0]',       

In [ ]:
model.fit(x=[input_ids,token_type_ids,attention_masks],
          y=y_train_new,
          batch_size=16,
          epochs=25)

Epoch 1/25
7/7 [==============================] - 14s 865ms/step - loss: 1.2578 - Accuracy: 0.4800
Epoch 2/25
7/7 [==============================] - 6s 863ms/step - loss: 0.7232 - Accuracy: 0.4600
Epoch 3/25
7/7 [==============================] - 6s 868ms/step - loss: 0.7655 - Accuracy: 0.4400
Epoch 4/25
7/7 [==============================] - 6s 873ms/step - loss: 1.3952 - Accuracy: 0.5600
Epoch 5/25
7/7 [==============================] - 6s 873ms/step - loss: 1.5628 - Accuracy: 0.5800
Epoch 6/25
7/7 [==============================] - 6s 878ms/step - loss: 0.8067 - Accuracy: 0.5000
Epoch 7/25
7/7 [==============================] - 6s 883ms/step - loss: 0.7989 - Accuracy: 0.5200
Epoch 8/25
7/7 [==============================] - 6s 887ms/step - loss: 5.5308 - Accuracy: 0.5200
Epoch 9/25
7/7 [==============================] - 6s 888ms/step - loss: 5.3062 - Accuracy: 0.5600
Epoch 10/25
7/7 [==============================] - 6s 888ms/step - loss: 1.9547 - Accuracy: 0.4600
Epoch 11/25
7/7 [=

In [ ]:
print(input_ids[1])

[  101     1   194  1153   194  8255    78   228     5     6  1463  4369
  5012   134    26     4   715     8   118  1634    14   394    20    13
   119   954   189   102     5   207   110  3103    21    14    69   188
     8    30    23     7     4   249   126    93     4   114     9  2300
  1523     5   647     4   116     9    35  8163     4   229     9   340
  1322     4   118     9     4   130  4901    19     4  1002     5    89
    29   952    46    37     4   455     9    45    43    38  1543  1905
   398     4  1649    26  6853     5   163    11  3215 10156     4  1153
     9   194   775     7  8255 11596   349  2637   148   605 15358  8003
    15   123   125    68 23141  6853    15   349   165  4362    98     5
     4   228     9    43 36893  1157    15   299   120     5   120   174
    11   220   175   136    50     9  4373   228  8255     5 25249   656
   245  2350     5     4  9837   131   152   491    18 46151    32  7464
  1212    14     9     6   371    78    22   625   

In [ ]:
model.evaluate([input_ids,token_type_ids,attention_masks], y_test_new)

4/4 [==============================] - 2s 470ms/step - loss: 0.6948 - Accuracy: 0.5300


[0.694765567779541, 0.5299999713897705]

In [ ]:
model.predict([input_ids,token_type_ids,attention_masks])

4/4 [==============================] - 2s 678ms/step


array([[0.42893285, 0.57106715],
       [0.42893288, 0.5710671 ],
       [0.42893282, 0.57106715],
       [0.42893285, 0.57106715],
       [0.4289328 , 0.57106715],
       [0.42893285, 0.57106715],
       [0.42893285, 0.57106715],
       [0.4289329 , 0.5710671 ],
       [0.42893288, 0.5710671 ],
       [0.42893285, 0.57106715],
       [0.42893285, 0.57106715],
       [0.42893288, 0.5710671 ],
       [0.42893282, 0.57106715],
       [0.4289328 , 0.57106715],
       [0.42893288, 0.5710671 ],
       [0.42893288, 0.5710671 ],
       [0.42893288, 0.5710671 ],
       [0.42893282, 0.57106715],
       [0.42893285, 0.57106715],
       [0.42893285, 0.57106715],
       [0.42893285, 0.57106715],
       [0.42893285, 0.57106715],
       [0.42893285, 0.57106715],
       [0.42893285, 0.57106715],
       [0.42893285, 0.57106715],
       [0.42893288, 0.5710671 ],
       [0.42893285, 0.57106715],
       [0.42893293, 0.5710671 ],
       [0.42893285, 0.57106715],
       [0.42893285, 0.57106715],
       [0.